# Agent Basics
- Agents
  - are a combimation of LLMs and code
  - LLMs reson about what steps to take and call for actions

- Agent loop
  - Choose a tool to use
  - Observe the output of the tool
  - Repeat until a stopping condition is met

- Stopping conditions can be:
  - LLM determined
  - Hardcoded rules



## In this lab:
- Build some tools
- Write our own agent loopu using LCEL
- Utilize `agent_executor` which:
  - Implements the agent loop
  - Adds error handling, early stopping, tracing, etc.

In [2]:
!pip install openai
!pip install langchain
!pip install pydantic==1.10.10

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.3/320.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 19.0 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24

In [3]:
import openai
from google.colab import userdata
key = userdata.get('OPENAI_API_KEY')

In [4]:
from langchain.tools import tool

In [5]:
import requests
from pydantic import BaseModel, Field
import datetime

# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""

    BASE_URL = "https://api.open-meteo.com/v1/forecast"

    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)

    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']

    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]

    return f'The current temperature is {current_temperature}°C'

In [6]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=d04886cea1a4d60fec3e6ff72e3d8f8c87bf90655e8d93c6d0942dd017354e7d
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [7]:
import wikipedia

@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [9]:
tools = [get_current_temperature, search_wikipedia]

In [14]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.tools.render import format_tool_to_openai_function
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [15]:
functions = [convert_to_openai_function(f) for f in tools]
model = ChatOpenAI(openai_api_key = key, temperature = 0).bind(functions = functions)

prompt = ChatPromptTemplate.from_messages([
    ('system', "you are helpful but sassy asistant"),
    ('human', '{input}')
])

chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [31]:
result = chain.invoke({'input': "What is langchain"})

In [32]:
result.tool

'search_wikipedia'

In [33]:
result.tool_input

{'query': 'langchain'}

In [37]:
result = chain.invoke({'input': "What is the weather in Pokhara?"})

In [38]:
result.tool

'get_current_temperature'

In [39]:
result.tool_input

{'latitude': 28.2639, 'longitude': 83.9722}

In [40]:
from langchain.prompts import MessagesPlaceholder


prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("human", "{input}"),

    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [41]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [49]:
reslut1 = chain.invoke({
    "input": 'What is the weather in Pokhara city',
    'agent_scratchpad': []
})

In [63]:
reslut1

AgentActionMessageLog(tool='get_current_temperature', tool_input={'latitude': 28.2639, 'longitude': 83.9722}, log="\nInvoking: `get_current_temperature` with `{'latitude': 28.2639, 'longitude': 83.9722}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":28.2639,"longitude":83.9722}', 'name': 'get_current_temperature'}}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 138, 'total_tokens': 163}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-15a9e0ff-7a34-4660-8926-96d3a4e90802-0')])

In [50]:
print(reslut1.tool); reslut1.tool_input

get_current_temperature


{'latitude': 28.2639, 'longitude': 83.9722}

In [51]:
observation = get_current_temperature(reslut1.tool_input)

In [52]:
observation

'The current temperature is 22.8°C'

In [53]:
type(reslut1)

langchain_core.agents.AgentActionMessageLog

In [54]:
from langchain.agents.format_scratchpad import format_to_openai_functions

In [55]:
reslut1.message_log

[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":28.2639,"longitude":83.9722}', 'name': 'get_current_temperature'}}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 138, 'total_tokens': 163}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-15a9e0ff-7a34-4660-8926-96d3a4e90802-0')]

In [59]:
format_to_openai_functions([(reslut1, observation),])

[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":28.2639,"longitude":83.9722}', 'name': 'get_current_temperature'}}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 138, 'total_tokens': 163}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-15a9e0ff-7a34-4660-8926-96d3a4e90802-0'),
 FunctionMessage(content='The current temperature is 22.8°C', name='get_current_temperature')]

In [60]:
format_to_openai_functions([(reslut1, observation)])

[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":28.2639,"longitude":83.9722}', 'name': 'get_current_temperature'}}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 138, 'total_tokens': 163}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-15a9e0ff-7a34-4660-8926-96d3a4e90802-0'),
 FunctionMessage(content='The current temperature is 22.8°C', name='get_current_temperature')]

In [61]:
result2 = chain.invoke({
    'input': 'what is the weather in Pokhara city',
    'agent_scratchpad': format_to_openai_functions([(reslut1, observation)])
})

In [62]:
result2

AgentFinish(return_values={'output': 'The current temperature in Pokhara city is 22.8°C.'}, log='The current temperature in Pokhara city is 22.8°C.')

In [64]:
from langchain.schema.agent import AgentFinish
def run_agent(user_input):
  intermediate_steps = []
  while True:
    result = chain.invoke({
        "input": user_input,
        "agent_scratchpad": format_to_openai_functions(intermediate_steps)
    })

    if isinstance(result, AgentFinish):
      return result

    tool = {
        "search_wikipedia": search_wikipedia,
        "get_current_temperature": get_current_temperature,
    }[result.tool]


    observation = tool.run(result.tool_input)

    intermediate_steps.append((result, observation))

In [65]:
from langchain.schema.runnable import RunnablePassthrough
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | chain

In [66]:
def run_agent(user_input):
    intermediate_steps = []
    while True:
        result = agent_chain.invoke({
            "input": user_input,
            "intermediate_steps": intermediate_steps
        })
        if isinstance(result, AgentFinish):
            return result
        tool = {
            "search_wikipedia": search_wikipedia,
            "get_current_temperature": get_current_temperature,
        }[result.tool]
        observation = tool.run(result.tool_input)
        intermediate_steps.append((result, observation))

In [67]:
run_agent("what is the weather is sf?")

AgentFinish(return_values={'output': 'The current temperature in San Francisco is 10.4°C.'}, log='The current temperature in San Francisco is 10.4°C.')

In [72]:
run_agent('what is Dell')

AgentFinish(return_values={'output': 'Dell Inc. is an American technology company that develops, sells, repairs, and supports computers and related products and services. Dell is known for its supply chain management and direct-to-customer sales approach. It sells personal computers, servers, data storage devices, network switches, software, and more. Dell is a subsidiary of Dell Technologies, a publicly traded company, and is ranked 31st on the Fortune 500 list in 2022.'}, log='Dell Inc. is an American technology company that develops, sells, repairs, and supports computers and related products and services. Dell is known for its supply chain management and direct-to-customer sales approach. It sells personal computers, servers, data storage devices, network switches, software, and more. Dell is a subsidiary of Dell Technologies, a publicly traded company, and is ranked 31st on the Fortune 500 list in 2022.')

In [71]:
run_agent("what is the this")

AgentFinish(return_values={'output': 'I\'m not sure what "this" you\'re referring to. Can you provide more context or clarify your question?'}, log='I\'m not sure what "this" you\'re referring to. Can you provide more context or clarify your question?')

In [73]:
run_agent('What\'s up?')

AgentFinish(return_values={'output': 'Not much, just here to assist you. What can I help you with today?'}, log='Not much, just here to assist you. What can I help you with today?')

In [74]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True)

In [78]:
agent_executor.invoke({"input": "what are Mobile Phone devices?"})



> Entering new AgentExecutor chain...

Invoking: `search_wikipedia` with `{'query': 'Mobile Phone devices'}`


Page: Windows Phone
Summary: Windows Phone (WP) is a discontinued mobile operating system developed by Microsoft for smartphones as the replacement successor to Windows Mobile and Zune. Windows Phone featured a new user interface derived from the Metro design language. Unlike Windows Mobile, it was primarily aimed at the consumer market rather than the enterprise market.
It was first launched in October 2010 with Windows Phone 7. Windows Phone 8 succeeded it in 2012, replacing the Windows CE-based kernel of Windows Phone 7 with the Windows NT kernel used by the PC versions of Windows (and, in particular, a large amount of internal components from Windows 8). Due to these changes, the OS was incompatible with all existing Windows Phone 7 devices, although it still supported apps originally developed for Windows Phone 7. In 2014, Microsoft released the Windows Phone 8.1 update

{'input': 'what is Mobile Phone devices?',
 'output': 'Mobile Phone devices refer to handheld computers that are small enough to hold and operate in the hand. These devices are typically battery-powered and have a flat-panel display along with built-in input devices like touchscreens or keypads. Modern mobile devices emphasize wireless networking for connectivity to the Internet and other devices through technologies like Wi-Fi, Bluetooth, cellular networks, or near-field communication.'}

In [76]:
agent_executor.invoke({"input": "what is the this?"})



> Entering new AgentExecutor chain...
I'm an assistant here to help you with your questions and tasks. How can I assist you today?

> Finished chain.


{'input': 'what is the this?',
 'output': "I'm an assistant here to help you with your questions and tasks. How can I assist you today?"}

In [77]:
agent_executor.invoke({"input": "what is the weather at Pokhara"})



> Entering new AgentExecutor chain...

Invoking: `get_current_temperature` with `{'latitude': 28.2639, 'longitude': 83.9722}`


The current temperature is 22.8°CThe current temperature in Pokhara is 22.8°C.

> Finished chain.


{'input': 'what is the weather at Pokhara',
 'output': 'The current temperature in Pokhara is 22.8°C.'}

In [79]:
agent_executor.invoke({"input": "Ram is the one who eats all the bananas in Byakaran books."})



> Entering new AgentExecutor chain...
Hmm, I'm not sure what you mean by "Byakaran books." Could you provide more context or clarify your statement?

> Finished chain.


{'input': 'Ram is the one who eats all the bananas in Byakaran books.',
 'output': 'Hmm, I\'m not sure what you mean by "Byakaran books." Could you provide more context or clarify your statement?'}

In [81]:
agent_executor.invoke({"input": "Who is Ram?"})



> Entering new AgentExecutor chain...

Invoking: `search_wikipedia` with `{'query': 'Ram'}`




/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


NameError: name 'self' is not defined

In [82]:
agent_executor.invoke({"input": "Ram brother name is Shyam"})



> Entering new AgentExecutor chain...
That's good to know! How can I assist you further?

> Finished chain.


{'input': 'Ram brother name is Shyam',
 'output': "That's good to know! How can I assist you further?"}

In [83]:
agent_executor.invoke({"input": "What is the name of Shyam's brother?"})



> Entering new AgentExecutor chain...
I'm not sure about Shyam's brother. Would you like me to look it up for you?

> Finished chain.


{'input': "What is the name of Shyam's brother?",
 'output': "I'm not sure about Shyam's brother. Would you like me to look it up for you?"}

In [84]:
##No history recordeddd

In [86]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [87]:
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | prompt | model | OpenAIFunctionsAgentOutputParser()

In [88]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

In [89]:
agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True, memory=memory)

In [90]:
agent_executor.invoke({"input": "Ram brother name is Shyam"})



> Entering new AgentExecutor chain...
That's good to know! How can I assist you further?

> Finished chain.


{'input': 'Ram brother name is Shyam',
 'chat_history': [HumanMessage(content='Ram brother name is Shyam'),
  AIMessage(content="That's good to know! How can I assist you further?")],
 'output': "That's good to know! How can I assist you further?"}

In [91]:
agent_executor.invoke({"input": "What is the name of Shyam's brother?"})



> Entering new AgentExecutor chain...
Shyam's brother's name is Ram.

> Finished chain.


{'input': "What is the name of Shyam's brother?",
 'chat_history': [HumanMessage(content='Ram brother name is Shyam'),
  AIMessage(content="That's good to know! How can I assist you further?"),
  HumanMessage(content="What is the name of Shyam's brother?"),
  AIMessage(content="Shyam's brother's name is Ram.")],
 'output': "Shyam's brother's name is Ram."}

# Creating the chatbotttt

In [94]:
@tool
def techResolve(query: str) -> str:
  "This fucntion gives you roadmap on the techstack you want to learn"
  print(type(query))
  return query[::-1]

In [95]:
tools = [get_current_temperature, search_wikipedia, techResolve]

In [98]:
import panel as pn  # GUI
pn.extension()
import panel as pn
import param

class cbfs(param.Parameterized):

    def __init__(self, tools, **params):
        super(cbfs, self).__init__( **params)
        self.panels = []
        self.functions = [format_tool_to_openai_function(f) for f in tools]
        self.model = ChatOpenAI(openai_api_key = key, temperature=0).bind(functions=self.functions)
        self.memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", "You are helpful but sassy assistant"),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])
        self.chain = RunnablePassthrough.assign(
            agent_scratchpad = lambda x: format_to_openai_functions(x["intermediate_steps"])
        ) | self.prompt | self.model | OpenAIFunctionsAgentOutputParser()
        self.qa = AgentExecutor(agent=self.chain, tools=tools, verbose=False, memory=self.memory)

    def convchain(self, query):
        if not query:
            return
        inp.value = ''
        result = self.qa.invoke({"input": query})
        self.answer = result['output']
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=450)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=450, styles={'background-color': '#F6F6F6'}))
        ])
        return pn.WidgetBox(*self.panels, scroll=True)


    def clr_history(self,count=0):
        self.chat_history = []
        return

In [100]:
cb = cbfs(tools)

inp = pn.widgets.TextInput( placeholder='Enter text here…')

conversation = pn.bind(cb.convchain, inp)

tab1 = pn.Column(
    pn.layout.Divider(),
    pn.panel(conversation,  loading_indicator=True, height=400),
    pn.layout.Divider(),
    pn.Row(inp),
)

dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# QnA_Bot')),
    pn.Tabs(('Conversation', tab1))
)
dashboard

Column
    [0] Row
        [0] Markdown(str)
    [1] Tabs
        [0] Column
            [0] Divider()
            [1] ParamFunction(function, _pane=Str, defer_load=False, height=400, loading_indicator=True)
            [2] Divider()
            [3] Row
                [0] TextInput(placeholder='Enter text here…')